In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys
import concurrent.futures
pd.set_option("display.max_rows",100)

In [49]:
secim_tarihi = pd.to_datetime('31.03.2024 00:00')

/var/folders/m4/z6n7r3kj74v5g05dqrb0cqbw0000gn/T/ipykernel_27737/545731946.py:1: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  secim_tarihi = pd.to_datetime('31.03.2024 00:00')


In [34]:
def scrape_page(page_link):
    r = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    entries_block = soup.find("ul", {"id": "entry-item-list"}).find_all("li")
    page_entries = []
    for entry in entries_block:
        content = entry.find("div", {"class": "content"}).text
        date = entry.find("a", {"class": "entry-date permalink"}).text
        page_entries.append((content, date))
    
    return page_entries

In [47]:
df = pd.read_csv("Eksisozluk-Entries.csv")

In [35]:
mainLink = "https://eksisozluk.com/31-mart-2024-yerel-secimleri--5927416" 
mainLink = mainLink.split("?")[0] 
    
headers = {
    "user-agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36 OPR/84.0.4316.31")
}

    
r = requests.get(mainLink, headers=headers)
soup = BeautifulSoup(r.content, "html.parser")
pageCount = int(soup.find("div", {"class": "pager"}).get("data-pagecount")) + 1

    
all_entries = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    page_links = [mainLink + "?p=" + str(i) for i in range(1, pageCount)]
    results = executor.map(scrape_page, page_links)
    for result in results:
        all_entries.extend(result)

df1 = pd.DataFrame(all_entries, columns=["comment", "date"])


    

In [58]:
df1["date"] = pd.to_datetime(df1["date"])

ValueError: time data "23.06.2019 19:33 ~ 24.06.2019 02:11" doesn't match format "%m.%d.%Y %H:%M", at position 7. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [51]:
def check_date(date_str):
    if '~' in date_str:
        dates = date_str.split('~')
        return pd.to_datetime(dates[0]) > secim_tarihi
    else:
        return pd.to_datetime(date_str) > secim_tarihi

In [52]:
df1['secimden_sonra'] = df1['date'].apply(lambda x: 1 if check_date(str(x)) else 0)

/var/folders/m4/z6n7r3kj74v5g05dqrb0cqbw0000gn/T/ipykernel_27737/2417164282.py:4: UserWarning: Parsing dates in %d.%m.%Y %H:%M  format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(dates[0]) > secim_tarihi
/var/folders/m4/z6n7r3kj74v5g05dqrb0cqbw0000gn/T/ipykernel_27737/2417164282.py:6: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(date_str) > secim_tarihi


In [42]:
df = pd.concat([df1, df], ignore_index=True)

In [44]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)

In [57]:
df.to_csv("Eksisozluk-Entries.csv", encoding="utf-8-sig", index=False)